In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import os
import copy #Để lấy được bản sao độc lập
######################################################################

''' Khởi tạo file Excel nếu chưa tồn tại
============================================'''
#--------------------------------------------------------------------------
#Dự tính:
columns_label=('Mã gạo', 'Tên gạo', 'Loại gạo', 'Giống gạo', 'Đặc tính', 'Khối lượng', 
                 'Thương hiệu', 'Giá chính thức', 'Giá khuyến mãi', 'Số lượng nhập', 'Số lượng xuất','Số lượng tồn')

#Thực hiện:
if not os.path.exists('gao_data.xlsx'):
    df = pd.DataFrame(columns=columns_label)
    df.to_excel('gao_data.xlsx', index=False)
#--------------------------------------------------------------------------

''' =============================Tạo Class phía dưới================================='''
''' Định nghĩa class Gạo
========================='''
class Gao:
    def __init__(self, ma_gao='', ten_gao='', loai_gao='', giong_gao='', dac_tinh='', khoi_luong='', 
                 thuong_hieu='', gia_chinh_thuc=0, gia_khuyen_mai=0, so_luong_nhap=0, so_luong_xuat=0):
        self.ma_gao = ma_gao                                        #01
        self.ten_gao= ten_gao                                       #02
        self.loai_gao = loai_gao                                    #03
        self.giong_gao = giong_gao                                  #04
        self.dac_tinh = dac_tinh                                    #05
        self.khoi_luong = khoi_luong                                #06
        self.thuong_hieu = thuong_hieu                              #07
        self.gia_chinh_thuc = gia_chinh_thuc                        #08
        self.gia_khuyen_mai = gia_khuyen_mai                        #09
        self.so_luong_nhap = so_luong_nhap                          #10
        self.so_luong_xuat = so_luong_xuat                          #11
        self.so_luong_ton = self.so_luong_nhap - self.so_luong_xuat #12

    def to_tuple(self):# Đổi thuộc tính thành tuple để dể treeview
        return (self.ma_gao, self.ten_gao, self.loai_gao, self.giong_gao, self.dac_tinh, self.khoi_luong,
                self.thuong_hieu, self.gia_chinh_thuc, self.gia_khuyen_mai, self.so_luong_nhap, 
                self.so_luong_xuat, self.so_luong_ton)
        
# Tạo Class quản lý danh sách đối tượng thông qua treeview và file excel:
'''======================================================================'''
class GaoManager:
    def __init__(self, treeview, file_path='gao_data.xlsx'):
        self.treeview = treeview #Nhận treeview
        self.file_path = file_path #nhận file liên quan
        self.gao_list = [] #tạo danh sách lưu các đối tượng
        self.load_data()
        
    def clear_treeview(self): # Hàm xóa sạch các dòng trên Treeview
        for item in self.treeview.get_children():
            self.treeview.delete(item)
            
    def load_treeview(self): # Hàm load dữ liệu lên Treeview
        self.clear_treeview()
        for gao in self.gao_list:
            self.treeview.insert('', 'end', values=gao.to_tuple())
        self.treeview.yview_moveto(1.0) # Tự động kéo thanh cuộn dọc xuống vị trí thấp nhất
   
    def load_data(self): #load dữ liệu nếu có file, chưa có file thì tạo file để chuẩn bị quản lý Gạo.
        if os.path.exists(self.file_path): #kiểm tra sự tồn tại của file excel
            df = pd.read_excel(self.file_path) #Nếu có file thì đổ ra DataFrame
            for _, row in df.iterrows(): # Duyệt từng dòng                
                gao = Gao(*[row[columns_label[i]] for i in range(len(columns_label)-1)])# Biến từng dòng thành Object Gạo tương ứng.
                self.gao_list.append(gao) #Thêm Gạo vào danh sách quản lý gao_list
                self.treeview.insert('', 'end', values=gao.to_tuple()) #Chuyển Gạo thành tuple để chèn vào cuối treeview
                self.treeview.yview_moveto(1.0)
        else:# Nếu không có file thì tạo DataFrame
            df = pd.DataFrame(columns=columns_label) #Tạo DataFrame
            df.to_excel(self.file_path, index=False) #Lưu Cấu trúc gồm các cột vào excel để lưu dữ liệu về sau.

    def add_gao(self, gao):#Thêm gạo = thêm vô treeview + thêm vô excel
        self.gao_list.append(gao)#Thêm gạo vào list quản lý
        self.treeview.insert('', 'end', values=gao.to_tuple())#Đẩy lên cuối dòng treeview
        self.save_data() #Lưu dữ liệu

    def save_data(self):#Chuyển các Object Gạo thành danh sách các tuple tương ứng (để có dạng DataFrame)
        data = [gao.to_tuple() for gao in self.gao_list]
        df = pd.DataFrame(data, columns=columns_label) #Chuyển DataFrame
        df.to_excel(self.file_path, index=False)
#1*
    def nhap(self):
        ma_gao = entries_thuoc_tinh[0].var.get()
        ten_gao= entries_thuoc_tinh[1].var.get()
        ton_tai=False #Kiểm tra sản phẩm tồn tại chưa
        for gao in self.gao_list:
            if gao.ma_gao == ma_gao or gao.ten_gao == ten_gao:
                ton_tai=True
                break
        if ton_tai:
            messagebox.showwarning("Cảnh báo", "Sản phẩm đã tồn tại! Vui lòng chọn nút sửa để cập nhật.")
            return #ktra xong. Kiểm tra entry nào thiếu phải báo điền đủ.
        else:
            clean=False #Kiểm tra có ô nào trống không
            for i in range(7):
                if entries_thuoc_tinh[i].var.get()=='':
                    clean=True
                    break
            if clean:
                messagebox.showwarning("Cảnh báo", f"Ô nhập thứ {i+1} bị trống, cần phải nhập liệu đầy đủ.")
                return
            else:
                if entries_thuoc_tinh[7].var.get()<1000:
                    messagebox.showwarning("Cảnh báo", "Giá bán đã nhập bé hơn 1000 đồng. Không hợp lý.")
                    return
                else:
                    if entries_thuoc_tinh[9].var.get()<0:
                        messagebox.showwarning("Cảnh báo", "Số lượng không thể nhỏ hơn 0 được.")
                        return
                    else:
                        new_gao = Gao(
                            ma_gao=ma_gao.upper(),
                            ten_gao=ten_gao,
                            loai_gao=entries_thuoc_tinh[2].var.get().upper(),
                            giong_gao=entries_thuoc_tinh[3].var.get(),
                            dac_tinh=entries_thuoc_tinh[4].var.get(),
                            khoi_luong=entries_thuoc_tinh[5].var.get(),
                            thuong_hieu=entries_thuoc_tinh[6].var.get().upper(),
                            gia_chinh_thuc=entries_thuoc_tinh[7].var.get(),
                            gia_khuyen_mai=entries_thuoc_tinh[8].var.get(),
                            so_luong_nhap=entries_thuoc_tinh[9].var.get()
                        )                        
                        self.add_gao(new_gao)
                        messagebox.showinfo("Thông báo", "Đã nhập sản phẩm thành công!")
                        return
#2*    
    def tim_kiem(self):
        # pass
        ''' is here.
        1. tạo list_search rỗng
        2. kiểm tra ma_gao có trong gao_list không:
            2a. Nếu có thì làm sạch treeview và đưa sản phẩm có mã gạo lên treevew
            2b. Nếu không thì duyệt từng entry
                    nếu entry có thông tin trong đối tượng nào thì chép đối tượng đó vào lit_search
                    nếu entry sau có thông tin trong list_search thì bỏ qua
                Sau khi duyệt xong các Entry thì kiểm tra list_search có khác rỗng không.
                    Nếu = rỗng thì báo ra màn hình là không có sản phẩm nào phù hợp thông tin của bạn.
                    Nếu != rỗng thì đưa toàn bộ thông tin trong list_search lên treeview.
                    Đưa ra thông báo bấm Reset để trở lại tình trạng trước khi tìm kiếm.
        ''' 
        list_search=[] #1
        ma_gao = entries_thuoc_tinh[0].var.get()
        if ma_gao !='': #2a
            for gao in self.gao_list:
                if gao.ma_gao==ma_gao:
                    self.clear_treeview()
                    self.treeview.insert('', 'end', values=gao.to_tuple())
                    messagebox.showinfo("Thông báo", "Để trở về màn hình trước khi tìm kiếm, bạn bấm nút Reset!")
                    return #ok
        else: #2b
            for gao in self.gao_list:
                do_tim = False
                for idx, entry in enumerate(entries_thuoc_tinh):
                    if entry.var.get():
                        words_entry=entry.var.get().lower().split()
                        gao_tuple = gao.to_tuple()[idx].lower()
                        for word in words_entry:
                            if word in gao_tuple:
                                do_tim=True
                                break
                    if do_tim:
                        break
                if do_tim:
                    list_search.append(gao)
            if list_search:
                self.clear_treeview()
                for gao in list_search:
                    self.treeview.insert('', 'end', values=gao.to_tuple())
                self.treeview.yview_moveto(1.0)
                messagebox.showinfo("Thông báo", "Để trở về màn hình trước khi tìm kiếm, bạn bấm nút Reset!")
            else:
                messagebox.showinfo("Thông báo", "Không có sản phẩm nào phù hợp thông tin của bạn.")
#3*
    def reset(self):
        for entry in entries_thuoc_tinh:
            entry.var.set('')
        self.load_treeview()            
#4*    
    # def xuat(gao):
    #     pass
    def xuat(self):#hàm chỉ chấp nhận xuất theo mã gạo, mọi thông tin khác có thể search để biết mã gạo
        ma_gao = entries_thuoc_tinh[0].var.get()  # Lấy mã gạo từ entry
        so_luong_xuat = int(entries_thuoc_tinh[9].var.get())  # Lấy số lượng xuất từ entry

        if not ma_gao:#Nếu ma_gao rỗng (chưa điền mã gạo)
            messagebox.showinfo("Thông báo", "Chỉ xuất theo mã gạo, vui lòng điền mã gạo.")
            return

        gao_tim_thay = None #khởi động biến kiểm tra

        for gao in self.gao_list:
            if gao.ma_gao == ma_gao:
                gao_tim_thay = gao# tìm thấy lấy luôn
                break #lấy rồi thì dừng

        if not gao_tim_thay:#nếu gạo tìm thấy thì if này bị bỏ qua
            messagebox.showinfo("Thông báo", "Không tìm thấy mã gạo trong danh sách.")
            return

        if so_luong_xuat > gao_tim_thay.so_luong_ton:
            messagebox.showinfo("Thông báo", f"Không đủ gạo tồn kho để xuất. Số lượng tồn kho hiện tại: {gao_tim_thay.so_luong_ton}")
            return

        gao_tim_thay.so_luong_xuat += so_luong_xuat
        gao_tim_thay.so_luong_ton -= so_luong_xuat

        # Cập nhật treeview
        self.clear_treeview()
        self.treeview.insert('', 'end', values=gao_tim_thay.to_tuple())

        # Lưu dữ liệu vào file
        self.save_data()

        messagebox.showinfo("Thông báo", "Để trở về màn hình trước khi xuất, bạn bấm nút Reset!")

#5*    
    def sua(self):
        # pass
        ma_gao = entries_thuoc_tinh[0].var.get()
        if not ma_gao:
            messagebox.showinfo("Thông báo", "Chỉ sửa khi đúng mã gạo, vui lòng điền mã gạo.")
            return
            
        #Tìm gạo đúng mã
        gao_tim_thay = None #khởi động biến kiểm tra
        for id_gao, gao in enumerate(self.gao_list):
            if gao.ma_gao == ma_gao:
                gao_tim_thay = copy.deepcopy(gao) # tạo bản sao độc lập
                break #lấy rồi thì ra
                
        if not gao_tim_thay:#nếu gạo tìm thấy thì if này bị bỏ qua
            messagebox.showinfo("Thông báo", "Không tìm thấy mã gạo trong danh sách.")
            return

        for id_ent, entry in enumerate(entries_thuoc_tinh):
            new=entry.var.get()
            if new: #tìm entry có dữ liệu để sửa, entry trống bỏ qua.
                if id_ent==0:
                   gao_tim_thay.ma_gao=new
                elif id_ent==1:
                    gao_tim_thay.ten_gao=new
                elif id_ent==2:
                    gao_tim_thay.loai_gao=new
                elif id_ent==3:
                    gao_tim_thay.giong_gao=new
                elif id_ent==4:
                    gao_tim_thay.dac_tinh=new
                elif id_ent==5:
                    gao_tim_thay.khoi_luong=new
                elif id_ent==6:
                    gao_tim_thay.thuong_hieu=new
                elif id_ent==7:
                    gao_tim_thay.gia_chinh_thuc=new
                elif id_ent==8:
                    gao_tim_thay.gia_khuyen_mai=new
                elif id_ent==9:
                    gao_tim_thay.so_luong_nhap=new
                    
        self.clear_treeview()
        self.treeview.insert('', 'end', values=gao.to_tuple())
        phan_ung = messagebox.askyesno("Cảnh báo", "Thông tin sẽ không thể khôi phục. Bạn có chắc muốn sửa không?")
        if phan_ung:
            self.gao_list[id_gao]=gao_tim_thay
            self.save_data()
            self.load_data()
            messagebox.showinfo("Thông báo", "Sản phẩm đã được cập nhật thành công.")
            return
#6*            
    def xoa(self):
        # pass
        ma_gao = entries_thuoc_tinh[0].var.get()
        if not ma_gao:
            messagebox.showinfo("Thông báo", "Chỉ xóa khi đúng mã gạo, vui lòng điền mã gạo.")
            return
            
        #Tìm gạo đúng mã
        gao_tim_thay = None #khởi động biến kiểm tra
        for id_gao, gao in enumerate(self.gao_list):
            if gao.ma_gao == ma_gao:
                gao_tim_thay = copy.deepcopy(gao) # tạo bản sao độc lập
                break #lấy rồi thì ra
                
        if not gao_tim_thay:#nếu gạo tìm thấy thì if này bị bỏ qua
            messagebox.showinfo("Thông báo", "Không tìm thấy mã gạo trong danh sách.")
            return

        phan_ung = messagebox.askyesno("Cảnh báo", "Thông tin sẽ không thể khôi phục. Bạn có chắc muốn xóa không?")
        if phan_ung:
            del self.gao_list[id_gao]
            self.save_data()
            self.load_data()
            messagebox.showinfo("Thông báo", "Sản phẩm đã được xóa thành công.")
            return        



### Định nghĩa class DoiTuong
'''=========================='''
class DoiTuong:
    def __init__(self,name=None, label=None, var=None,width=None,command=None):
        self.name=name
        self.label = label
        self.var = var
        self.width=width if width is not None else len(self.label)
        self.command=command
''' =============================Tạo Class phía trên================================='''


''' =============================Tạo HÀM phía dưới================================='''
''' Hàm cập nhật thay đổi của Frame đặc tính
------------------------------------------------------------------------'''
def update_dac_tinh(label):
    # Biến để lưu trữ text từ các checkbutton đã chọn
    selected_texts = []

    # Kiểm tra trạng thái của các Checkbutton
    for item in labels_dac_tinh:
        if item.var.get() == 1:
            selected_texts.append(item.label)

    # Cập nhật nội dung của entry thứ 4 trong entries_thuoc_tinh
    entries_thuoc_tinh[4].var.set(" ".join(selected_texts))
'''--------------------------------------------------------------------'''

### Hàm cập nhật thay đổi của Frame khối lượng
#-----------------------------------------------------------------------
def update_khoi_luong(selected_idx):
    # Đặt lại tất cả các Checkbutton khác về 0
    for idx, item in enumerate(labels_khoi_luong):
        if idx != selected_idx:
            item.var.set(0)
            
    # Biến để lưu trữ text từ Checkbutton đã chọn
    selected_text = ''

    # Kiểm tra trạng thái của các Checkbutton
    for item in labels_khoi_luong:
        if item.var.get() == 1:
            selected_text += item.label
    
    # Cập nhật nội dung của entry thứ 5 trong entries_thuoc_tinh
    entries_thuoc_tinh[5].var.set(selected_text)
#--------------------------------------------------------------------------
''' =============================Tạo HÀM phía trên================================='''




'''=========================================================================================
============================================================================================
============================================================================================
============================================================================================
Tạo cửa sổ chính ===========================================================================
============================================================================================'''
window = tk.Tk() # khởi tạo cửa sổ chính chứa trong biến window
window.config(bg="black") # màu nền
window.title("Quản lý gạo") # tiêu đề
window.geometry("+10+10") # cách trên 10px, cách trái 10px
window.minsize(800, 600) # kích thước tối thiểu cửa sổ
'''========================================================================================
===========================================================================================
===========================================================================================
============================================================================================
============================================================================================
============================================================================================
============================================================================================'''




''' =============================Tạo Treeview phía dưới================================='''
''' Tạo TreeView
-------------------------------------------------------------------------------'''
tree = ttk.Treeview(window, columns=columns_label, show='headings')

# Đặt tiêu đề cho các cột
for col in columns_label:
    tree.heading(col, text=col)
    if col=="Tên gạo" or col=="Thương hiệu":
        tree.column(col, width=200)
    else:
        tree.column(col, width=20)
    
# Gắn Treeview lên lưới window.
tree.grid(row=0, column=0, columnspan=12, sticky="news")

# Thêm scrollbar cho treeview
#cuộn dọc
v_scrollbar = ttk.Scrollbar(window, orient="vertical", command=tree.yview)
v_scrollbar.grid(row=0, column=12, sticky='ns')
tree.configure(yscrollcommand=v_scrollbar.set)
#cuộn ngang
h_scrollbar = ttk.Scrollbar(window, orient="horizontal", command=tree.xview)
h_scrollbar.grid(row=1, column=0, columnspan=12, sticky='ew')
tree.configure(xscrollcommand=h_scrollbar.set)

# Cấu hình cho lưới tự mở rộng
window.grid_rowconfigure(0, weight=1)# cho hàng 0
for idx in range(12):#cho các cột.
    if idx==1:
        window.grid_columnconfigure(idx, weight=3)
    else:
        window.grid_columnconfigure(idx, weight=1)

''' =============================Tạo treeview phía trên================================='''
### Gọi quản lý gạo (tải dữ liệu):
gao_manager = GaoManager(tree)

''' =============================Tạo Frame phía dưới================================='''
### Thiết kế Frame Đặc tính:
#-------------------------------------------------------------------------
# Tạo Frame Đặc tính:
frm_dac_tinh = tk.Frame(window, bg="light grey")
frm_dac_tinh.grid(row=4, column=4, padx=(2, 0), pady=0, sticky="wens")
# Tạo các label đặc tính
labels_dac_tinh = [
    DoiTuong(label="Nở xốp", var=tk.IntVar()),
    DoiTuong(label="Dẻo", var=tk.IntVar()),
    DoiTuong(label="Thơm", var=tk.IntVar()),
    DoiTuong(label="Mềm", var=tk.IntVar())]
# Gắn checkbutton và labels
for idx, item in enumerate(labels_dac_tinh):
    chk = tk.Checkbutton(frm_dac_tinh, bg="dark green", variable=item.var, 
                            command=lambda i=idx: update_dac_tinh(labels_dac_tinh[i].label_widget))
    chk.grid(row=0, column=idx, padx=(2, 2), pady=0, sticky="we")
    lbl = tk.Label(frm_dac_tinh, text=item.label, bg="dark green", fg="white")
    lbl.grid(row=1, column=idx, padx=(2, 2), pady=(0, 4), sticky="we")
    item.label_widget = lbl
    frm_dac_tinh.grid_columnconfigure(idx, weight=1)    
#--------------------------------------------------------------------------

''' Thiết kế Frame Khối lượng
----------------------------------------------------------------------------'''
# Tạo Frame khối lượng:
frm_khoi_luong = tk.Frame(window, bg="light grey")
frm_khoi_luong.grid(row=4, column=5, padx=(2, 0), pady=0, sticky="wens")
# Tạo các label khối lượng
labels_khoi_luong = [
    DoiTuong(label="1kg", var=tk.IntVar(value=0)),
    DoiTuong(label="2kg", var=tk.IntVar(value=0)),
    DoiTuong(label="5kg", var=tk.IntVar(value=0)),
    DoiTuong(label="khác", var=tk.IntVar(value=0))]
# Gắn Checkbutton và labels
for idx, item in enumerate(labels_khoi_luong):
    chk = tk.Checkbutton(frm_khoi_luong, bg="dark green", variable=item.var, 
                            command=lambda i=idx: update_khoi_luong(i))
    chk.grid(row=0, column=idx, padx=(2, 2), pady=0, sticky="we")
    lbl = tk.Label(frm_khoi_luong, text=item.label, bg="dark green", fg="white")
    lbl.grid(row=1, column=idx, padx=(2, 2), pady=(0, 4), sticky="we")
    item.label_widget = lbl
    frm_khoi_luong.grid_columnconfigure(idx, weight=1)
'''------------------------------------------------------------------------'''
''' =============================Tạo Frame phía Trên================================='''


''' =============================Tạo widget khác phía dưới================================='''
### Thiết kế dãy label cho thuộc tính:
'''=============================='''
#--------------------------------------------------------------------------
# Tạo list label thuộc tính:
labels_thuoc_tinh = [DoiTuong(label=columns_label[i]) for i in range(9)]+[DoiTuong(label="Số lượng")]
   
# Tạo widget và gắn lên window
for idx,label in enumerate(labels_thuoc_tinh):
    temp = tk.Label(window, text=label.label, width=label.width, bg="dark green",fg="light cyan",font=('Time New Roman',12,))
    temp.grid(row=3, column=idx, padx=(2, 0), pady=(2, 0), sticky="wens")
#----------------------------------------------------------------------------

''' Thiết kế dãy entry cho thuộc tính: 
===================================
-----------------------------------------------------------------------------'''
#Tạo list entry thuộc tính:
entries_thuoc_tinh = [DoiTuong(width=len(columns_label[i]),var=tk.StringVar()) for i in range(7)]
entries_thuoc_tinh +=[DoiTuong(width=len(columns_label[i]),var=tk.IntVar()) for i in range(7,10)]
#Tạo widget và gắn lên window
for i in range(10):
    temp = tk.Entry(window,textvariable=entries_thuoc_tinh[i].var, width=entries_thuoc_tinh[i].width,bg="light cyan")
    temp.grid(row=2, column=i, padx=(2, 0), pady=0, sticky="wens")
    entries_thuoc_tinh[i].entry_widget=temp
'''----------------------------------------------------------------------------'''

### Thiết kế các nút lệnh button:
'''========================='''
#----------------------------------------------------------------------------------
#Tạo list các nút lệnh:
list_buttons=[
    DoiTuong(label="Nhập", command=gao_manager.nhap),
    DoiTuong(label="Xuất", command=gao_manager.xuat),
    DoiTuong(label="Reset", command=gao_manager.reset),
    DoiTuong(label="Tìm kiếm", command=gao_manager.tim_kiem),
    DoiTuong(label="Sửa", command=gao_manager.sua),    
    DoiTuong(label="Xóa", command=gao_manager.xoa)
    
]
#Tạo và gắn các button lên window
for idx, button in enumerate(list_buttons):
        temp = tk.Button(window, text=button.label,command=button.command,bg="orange",width=len(list_buttons[3].label))
        temp.grid(row=idx%3+2, column=idx//3+10, padx=2, pady=0, sticky="wen")
#------------------------------------------------------------------------------------
''' =============================Tạo widget khác phía trên================================='''




''' Vùng thử nghiệm
___________________________________________________________________________________________________'''
# Textbox để hiển thị text từ label
#-----------------------------------------------------------
# Dùng Entry tạo textbox:
textbox = tk.Entry(window)
textbox.grid(row=5, column=4, columnspan=2, padx=(2, 2), pady=(10, 10), sticky="we")
# Biến để lưu trữ text đã chọn
selected_text = ""
#------------------------------------------------------------
'''___________________________________________________________________________________________________'''
##############################################################################################
# Bắt đầu vòng lặp sự kiện
window.mainloop()

# Khu vực phân tích lỗi:

In [111]:
for x in range (5):
    if x==3:        
        break
    print(x,'x')
    for y in range (5):
        if y==3:
            break    
        print(y,'y')
        for z in range(5):
            if z==3:                
                break
            print(z,'z')
    

0 x
0 y
0 z
1 z
2 z
1 y
0 z
1 z
2 z
2 y
0 z
1 z
2 z
1 x
0 y
0 z
1 z
2 z
1 y
0 z
1 z
2 z
2 y
0 z
1 z
2 z
2 x
0 y
0 z
1 z
2 z
1 y
0 z
1 z
2 z
2 y
0 z
1 z
2 z


# Lưu tạm:

In [ ]:

                
        # def tim_kiem(self):
        #     list_search = []
        #     ma_gao = entries_thuoc_tinh[0].var.get().upper()
        
        #     if ma_gao != '':
        #         for gao in self.gao_list:
        #             if gao.ma_gao == ma_gao:
        #                 self.clear_treeview()
        #                 self.treeview.insert('', 'end', values=gao.to_tuple())
        #                 self.show_message("Thông báo", "Để trở về màn hình trước khi tìm kiếm, bạn bấm nút Reset!")
        #                 return

        #     for gao in self.gao_list:
        #         match = True
        #         for idx, entry in enumerate(entries_thuoc_tinh):
        #             if entry.var.get() and entry.var.get().lower() not in gao.to_tuple()[idx].lower():
        #                 match = False
        #                 break
        #         if match and gao not in list_search:
        #             list_search.append(gao)

        #     if list_search:
        #         self.clear_treeview()
        #         for gao in list_search:
        #             self.treeview.insert('', 'end', values=gao.to_tuple())
        #         self.show_message("Thông báo", "Để trở về màn hình trước khi tìm kiếm, bạn bấm nút Reset!")
        #     else:
        #         self.show_message("Thông báo", "Không có sản phẩm nào phù hợp thông tin của bạn.")
            
                
